# Unsupervised Clustering - 4 Clusters

# Note
This is not the final version of this feature, for the more refined version go to "Multifeature Experiment/MultifeatureExperiment.ipynb"

### DataPoint
First, we'll have to create a DataPoint class, that can be used to store and compare input-datapoints. We'll do this as follows, including a ```dist``` function to compute the squared-distance between two datapoints:

In [1]:
# This import is required in python version < 3.11 to allos reference to DataPoint inside definition of DataPoint
from __future__ import annotations


# Helper functions
def sqr(x: float):
    return x * x


class DataPoint:
    x: float
    y: float
    cluster: int
    fidelity: float

    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
        self.cluster = 0
        self.fidelity = 0

    def dist(self, o: DataPoint) -> float:
        return sqr(o.x - self.x) + sqr(o.y - self.y)

Also implement the ```map``` function, that can be used to map Data Points onto the Bloch Sphere:

In [2]:
def map(a: float, b: float, c: float, d: float, e: float) -> float:
    return d + ((a - b)/(c - b)) * (e - d)

### Qiskit
Now, we start working with Qiskit! Import all the necessary elements:

In [3]:
import numpy as np

import matplotlib.pyplot as plt

from qiskit.providers.aer import Aer
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, DensityMatrix
from qiskit import transpile
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.optimizers import ADAM
from qiskit.algorithms.optimizers import GradientDescent
from qiskit.quantum_info import state_fidelity

from tqdm import tqdm
import math

from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
# Fix the np random seed
np.random.seed(12)

#
# Hard-code the data points that the algorithm will be performed on
#
x_range = iris.data[:, 1]
y_range = iris.data[:, 3]
data_points = []

min_x = 2
max_x = 4.4

min_y = 0.1
max_y = 2.5

alpha = 1
lamda = 0.1
"""
points_0 = [DataPoint(np.random.uniform(low=0, high=20),
                      np.random.uniform(low=0, high=20)) for i in range(10)]

points_1 = [DataPoint(np.random.uniform(low=30, high=50),
                      np.random.uniform(low=0, high=20)) for i in range(10)]

points_2 = [DataPoint(np.random.uniform(low=0, high=20),
                      np.random.uniform(low=30, high=50)) for i in range(10)]

points_3 = [DataPoint(np.random.uniform(low=30, high=50),
                      np.random.uniform(low=30, high=50)) for i in range(10)]
"""
for l in range(len(x_range)):
    data_points.append(DataPoint(x_range[l], y_range[l]))


# Plot the datapoints
plt.title("Input Data Points")
plt.scatter(
    [dp.x for dp in data_points],
    [dp.y for dp in data_points]
)
plt.show()

#
# Hard-code the two reference states on the bloch-sphere that we will use as references for two clusters
#
# [1, 0] = |0> and [0, 1] = |1>
reference_points = [
    Statevector([1, 0, 0, 0]), # 0-0
    Statevector([0, 1, 0, 0]), # 0-1
    Statevector([0, 0, 1, 0]), # 1-0
    #Statevector([0, 0, 0, 1]), # 1-1
]

circuit_counter = 0
# Function: setup the variational form according to given parameters
def setup_var_form(dp: DataPoint, params, qubits = 2) -> QuantumCircuit:
    global circuit_counter
    
    q_circuit = QuantumCircuit(qubits)
    q_circuit.name = "circuit" + str(circuit_counter)
    circuit_counter += 1
    
    # Initialize both qubits in state zero
    q_circuit.initialize([1, 0], 0)
    q_circuit.initialize([1, 0], 1)
    
    # Unitary transformations that insert data-point properties
    for i in range(qubits):
        x_map = map(dp.x, min_x, max_x, 0, math.pi)
        y_map = map(dp.y, min_y, max_y, 0, math.pi)
        q_circuit.u(x_map,
                    y_map, x_map * y_map, i) # data-point[x] on qubit 1 as x-transformation
    
    # Create entanglement between the two qubits
    #q_circuit.h(0)
    #q_circuit.cnot(0, 1)
    
    # Unitary transformations for optimization parameters
    for i in range(qubits-1):
        q_circuit.u(params[2*i], params[2*i+1], 0, i) # Parameters 0 and 1 on qubit 0
        q_circuit.cnot(i, i+1)
    q_circuit.u(params[2*qubits-2], params[2*qubits-1], 0, qubits-1) # Parameters 2 and 3 on qubit 1
    
    q_circuit.save_statevector()
    
#     print("Quantum Circuit:")
#     print(q_circuit)
    
    return q_circuit

#setup_variational_form(DataPoint(0, 5), [1, 2, 3, 4])



# Objective function
def objective(params) -> float:
    #print("Running objective...")
    
    total_cost = 0
    
    # Select the qiskit backend
    qiskit_backend = Aer.get_backend("aer_simulator")
    
    cache = []
    fidelities = [[None for i in range(len(data_points))] for j in range(len(reference_points))]
    centroids = [DataPoint(0, 0) for i in range(len(reference_points))]
    counts = [0 for i in range(len(reference_points))]
    
    for i in range(len(data_points)):
        cache.append(None)
    
    for i in tqdm(range(len(data_points))):
        #print("\tConsidering point [" + str(i) + ", " + str(j) + "]")
        # Compute the fidelity
        job_dp1 = cache[i]
                
        if job_dp1 == None:
            job_dp1 = qiskit_backend.run(
                transpile(
                    setup_var_form(data_points[i], params),
                    backend=qiskit_backend
                )
            ).result().get_statevector()
            cache[i] = job_dp1
                
            # Compute fidelities with reference points
            data_points[i].fidelity = 0
            for ref_index, r in enumerate(reference_points):
                # Add the input-set-distance to the cost
                    
                if (fidelities[ref_index][i] == None):
                    fidelities[ref_index][i] = state_fidelity(r, job_dp1)
                fidelity_state1 = fidelities[ref_index][i]
                    
                if (fidelity_state1 > data_points[i].fidelity):
                    data_points[i].cluster = ref_index
                    data_points[i].fidelity = fidelity_state1
                    
        penalty = 0
        for r in range(len(reference_points)):
            penalty += fidelities[r][i] - 1
        total_cost += penalty ** 2
    
    for dp in data_points:
        centroids[dp.cluster].x += dp.x
        centroids[dp.cluster].y += dp.y
        counts[dp.cluster] += 1
    
    for i in range(len(reference_points)):
        if (counts[i] > 0):
            centroids[i].x = centroids[i].x / counts[i]
            centroids[i].y = centroids[i].y / counts[i]
    
    for i in range(len(data_points)):
        centroid_distance = data_points[i].dist(centroids[data_points[i].cluster])
        for j in range(len(data_points)):
            cost = data_points[i].dist(data_points[j]) ** alpha
            cost += centroid_distance * lamda
            sum_fidels = 0
            for r in range(len(reference_points)):
                sum_fidels += (1-fidelities[r][i])*(1-fidelities[r][j])
            cost *= sum_fidels
            total_cost += cost
        
    print("Total cost: " + str(total_cost))
    return total_cost
            
# Select the right optimizer
optimizer = ADAM(maxiter=30, tol=10^-6, lr=0.05)
# optimizer = GradientDescent(maxiter=10)

# Initialize the parameters with random values
params_init = np.random.rand(4)

# Perform the optimization and store the result
optimal_params = optimizer.minimize(fun=objective, x0=params_init).x

print(optimal_params)
print("Total cost: " + str(objective(optimal_params)))

# Divide the points into clusters, according to the information that's stored within the datapoints
clusters = []

for ref_index in range(len(reference_points)):
    current_cluster = []
    for dp in data_points:
        if ref_index == dp.cluster:
            current_cluster.append(dp)
    clusters.append(current_cluster)
    
for c in clusters:
    plt.scatter(
        [dp.x for dp in c],
        [dp.y for dp in c]
    )
plt.title("Clusters after optimizing")
plt.show()

for i, c in enumerate(clusters):
    print("Data Points in cluster " + str(i) + ":")
    for dp in c:
        print("[" + str(dp.x) + ", " + str(dp.y) + "]")
